In [77]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from collections import Counter
import Bio.PDB.Polypeptide as pp
import networkx as nx
import biographs as bg
import os
from main_functions import *

In [48]:
## Sector positions of PDZ domain protein (1be9)
## 20% of total positions (17 out of 83)
sectors = {322, 323, 325, 327, 329, 330, 336, 345, 347, 350, 351, 353, 362, 364, 372, 375, 379}

In [22]:
## Functionally sensitive positions (McLaughlin, 2011), 2 STD below average
## 24% of total positions (20 out of 83)
funct = {323, 324, 325, 327, 328, 329, 330, 336, 338, 341, 347, 353, 359, 362, 367, 372, 375, 376, 379, 388}

In [32]:
functional_data = dict()
for protein in proteins:
    csv_file = os.path.join('data', f'functional_{protein}.csv')
    functional_data[protein] = pd.read_csv(csv_file, index_col=0, header=0)

In [21]:
## 40% of positions wiht lowest functional values, our FSPs (33 out of 83 positions)
fsp_40_aa = GetPercentage(0.4, 'lowest', functional_data['1be9'])
fsp_40 = set([int(pos[1:]) for pos in fsp_40_aa]) ## remove AA letter code

In [29]:
len(fsp_40.intersection(sectors))

13

In [30]:
len(funct.intersection(sectors))

12

In [36]:
## Our predictions maximizing precision
max_precision_aa = GetNetworkExtremes('1be9', 4, [1.5]*4)
max_precision = set([int(pos[1:]) for pos in max_precision_aa])

In [41]:
## Our predictions maximizing recall
max_recall_aa = GetNetworkExtremes('1be9', 2, [1]*4)
max_recall = set([int(pos[1:]) for pos in max_recall_aa])

In [42]:
## Our predictions maximizing both
max_both_aa = GetNetworkExtremes('1be9', 2, [1.5]*4)
max_both = set([int(pos[1:]) for pos in max_both_aa])

In [70]:
## Comparing our predictions to sectors instead of FSPs
predictions = {'Max Precision': max_precision, 'Max Recall': max_recall, 'Max both': max_both}
for prediction in predictions.keys():
    print(prediction)
    positions = predictions[prediction]
    precision = ToPercentage(positions.intersection(sectors), positions)
    recall = ToPercentage(positions.intersection(sectors), sectors)
    ssps = ToPercentage(positions, functional_data['1be9'].columns)
    improvement = ToPercentage(recall, ssps)
    print('Precision: ', precision)
    print('Recall: ', recall)
    print('SSPs: ', ssps)
    print('Improvement: ', round(improvement/100,2), '\n')

Max Precision
Precision:  33.3
Recall:  41.2
SSPs:  25.3
Improvement:  1.63 

Max Recall
Precision:  28.3
Recall:  76.5
SSPs:  55.4
Improvement:  1.38 

Max both
Precision:  31.6
Recall:  70.6
SSPs:  45.8
Improvement:  1.54 



In [71]:
## Comparing our predictions to 24% of FSPs (instead of 40%)
predictions = {'Max Precision': max_precision, 'Max Recall': max_recall, 'Max both': max_both}
for prediction in predictions.keys():
    print(prediction)
    positions = predictions[prediction]
    precision = ToPercentage(positions.intersection(funct), positions)
    recall = ToPercentage(positions.intersection(funct), funct)
    ssps = ToPercentage(positions, functional_data['1be9'].columns)
    improvement = ToPercentage(recall, ssps)
    print('Precision: ', precision)
    print('Recall: ', recall)
    print('SSPs: ', ssps)
    print('Improvement: ', round(improvement/100,2), '\n')

Max Precision
Precision:  42.9
Recall:  45.0
SSPs:  25.3
Improvement:  1.78 

Max Recall
Precision:  41.3
Recall:  95.0
SSPs:  55.4
Improvement:  1.72 

Max both
Precision:  44.7
Recall:  85.0
SSPs:  45.8
Improvement:  1.86 



In [72]:
## Comparing our predictions to 40% of FSPs
predictions = {'Max Precision': max_precision, 'Max Recall': max_recall, 'Max both': max_both}
for prediction in predictions.keys():
    print(prediction)
    positions = predictions[prediction]
    precision = ToPercentage(positions.intersection(fsp_40), positions)
    recall = ToPercentage(positions.intersection(fsp_40), fsp_40)
    ssps = ToPercentage(positions, functional_data['1be9'].columns)
    improvement = ToPercentage(recall, ssps)
    print('Precision: ', precision)
    print('Recall: ', recall)
    print('SSPs: ', ssps)
    print('Improvement: ', round(improvement/100,2), '\n')

Max Precision
Precision:  66.7
Recall:  42.4
SSPs:  25.3
Improvement:  1.68 

Max Recall
Precision:  60.9
Recall:  84.8
SSPs:  55.4
Improvement:  1.53 

Max both
Precision:  68.4
Recall:  78.8
SSPs:  45.8
Improvement:  1.72 



In [64]:
## Distribution of functional values across proteins

In [65]:
# Standardize functional data, store resulting dataframes in standardized_data
standardized_data = dict()
for protein in proteins:
    data = functional_data[protein]
    data_array = data.to_numpy()
    data_mean = np.nanmean(data, dtype=np.float64)
    data_std = np.nanstd(data, dtype=np.float64)
    data = data.apply(lambda x:(x-data_mean)/data_std)
    standardized_data[protein] = data

In [66]:
def CheckDistribution(cutoffs, proteins=proteins, data=standardized_data):
    """Return df with number of positions with mean above cutoffs (if positive), or below cutoffs (if negative) and 
    percentage. """
    columns = []
    for cutoff in cutoffs:
        columns.extend([f'{cutoff} std', f'% {cutoff} std'])
    values = pd.DataFrame(index=proteins, columns=columns)
    for protein in proteins:
        data_df = data[protein]
        data_mean = list(data_df.mean())
        n = len(data_mean)
        for cutoff in cutoffs:
            if cutoff > 0:
                m = len([i for i in data_mean if i > cutoff])
            elif cutoff < 0:
                m = len([i for i in data_mean if i < cutoff])
            values.at[protein, f'{cutoff} std'] = m
            values.at[protein, f'% {cutoff} std'] = str(round(100*m/n, 1))+'%'
    return values

In [74]:
CheckDistribution([-2, -1.5, -1, -0.5, 0.5, 1, 1.5, 2])

,-2 std,% -2 std,-1.5 std,% -1.5 std,-1 std,% -1 std,-0.5 std,% -0.5 std,0.5 std,% 0.5 std,1 std,% 1 std,1.5 std,% 1.5 std,2 std,% 2 std
1be9,3,3.6%,3,3.6%,11,13.3%,17,20.5%,14,16.9%,0,0.0%,0,0.0%,0,0.0%
1d5r,2,0.7%,13,4.2%,49,16.0%,78,25.4%,107,34.9%,0,0.0%,0,0.0%,0,0.0%
1nd4,0,0.0%,0,0.0%,0,0.0%,71,27.8%,61,23.9%,26,10.2%,6,2.4%,2,0.8%
3dqw,0,0.0%,0,0.0%,30,12.4%,68,28.1%,62,25.6%,19,7.9%,9,3.7%,4,1.7%
4bz3,0,0.0%,2,0.9%,34,14.7%,81,35.1%,89,38.5%,11,4.8%,0,0.0%,0,0.0%


In [78]:
## Density of WT networks 
sample_thresholds = [4.0, 6.0, 8.0, 10.0]
density = pd.DataFrame(index=proteins, columns=sample_thresholds)

In [81]:
for protein in proteins:
    original_prot = bg.Pmolecule(f"data/pdb/{protein}.pdb")
    for threshold in sample_thresholds:
        network = original_prot.network(cutoff=threshold)
        density.loc[protein, threshold] = nx.density(network)

In [82]:
density

,4.0,6.0,8.0,10.0
1be9,0.064846,0.102241,0.186555,0.268487
1d5r,0.025654,0.043176,0.078517,0.117519
1nd4,0.016064,0.025224,0.046265,0.069078
3dqw,0.007428,0.011943,0.022381,0.03461
4bz3,0.018005,0.030098,0.056286,0.086402


In [115]:
## Diameter at different thresholds 
for protein in proteins:
    print(protein)
    for thresh in [4.0, 6.0, 8.0, 10.0]:
        max_values = list(set(ReadNetworkCSV(protein, thresh, 'distance').max().to_list()))
        max_values.sort()
        print(max_values[0], max_values[-1])

1be9
2.0 8.0
4.0 10.0
4.0 10.0
4.0 9.0
1d5r
2.0 9.0
2.0 8.0
2.0 7.0
2.0 7.0
1nd4
2.0 9.0
2.0 7.0
2.0 5.5
2.0 5.5
3dqw
2.0 8.25
2.25 6.25
2.75 5.25
2.75 5.25
4bz3
2.0 10.0
4.0 10.0
5.0 10.0
6.0 10.5


In [114]:
## Nodes at different thresholds 
for protein in proteins:
    print(protein)
    for thresh in [4.0, 6.0, 8.0, 10.0]:
        max_values = list(set(ReadNetworkCSV(protein, thresh, 'nodes').max().to_list()))
        max_values.sort()
        print(max_values[0], max_values[-1])

1be9
11.0 43.0
19.0 68.0
29.0 88.0
41.0 114.0
1d5r
5.0 38.0
7.0 62.0
12.0 91.0
16.0 123.0
1nd4
4.5 36.5
6.5 59.0
11.0 96.5
15.0 118.5
3dqw
3.0 37.5
6.0 58.75
11.75 80.25
18.25 118.75
4bz3
27.5 55.5
56.0 94.0
93.0 149.5
137.0 209.0


In [116]:
## Comparing predictions including distance vs not including it 

In [117]:
def GetNetworkExtremesNoDist(protein, mincount, measure_cutoffs, thresh=9.0):
    # In case thresh is given as an int
    threshold = float(thresh)
    network_extremes_list = []
    for i, measure in enumerate(['nodes', 'edges', 'weight']):
        network_df = Standardize(protein, threshold, measure)
        if measure_cutoffs[i] > 0:
            extremes = network_df.columns[(network_df > measure_cutoffs[i]).any()].tolist()
        else:
            extremes = network_df.columns[(network_df < measure_cutoffs[i]).any()].tolist()
        network_extremes_list.extend(extremes)

    counter = Counter(network_extremes_list)
    positions = list(set(network_extremes_list))
    return set([pos for pos in positions if counter[pos] >= mincount])

In [136]:
def GetScores(functional_percentage, network_mincount, measure_cutoffs, thresh=9.0, mean=True):
    """Return precision, recall and prediction percentage scores. Returns list of lists, [precision, recall,
    predper] for each protein and the mean, if mean == True """
    which_scores = ['Precision', 'Recall', 'Improvement', 'SSPs']
    scores = {score:[] for score in which_scores}
    for protein in proteins:
        network_extremes = GetNetworkExtremes(protein, network_mincount, measure_cutoffs, thresh=thresh)
        funct_extremes = GetPercentage(functional_percentage, 'lowest', functional_data[protein])
        how_many = len(network_extremes.intersection(funct_extremes))
        positions = len(functional_data[protein].columns)
    
        scores['Recall'].append(ToPercentage(how_many, funct_extremes))
        precision_score = ToPercentage(how_many, network_extremes)
        scores['Precision'].append(precision_score)
        scores['Improvement'].append(round(precision_score/40, 2))
        scores['SSPs'].append(ToPercentage(network_extremes, positions))
        
    if mean:
        for score in which_scores:
            scores[score].append(round(np.mean(scores[score]), 2))
            
    return scores

In [137]:
def GetScoresNoDist(functional_percentage, network_mincount, measure_cutoffs, thresh=9.0, mean=True):
    """Return precision, recall and prediction percentage scores. Returns list of lists, [precision, recall,
    predper] for each protein and the mean, if mean == True """
    which_scores = ['Precision', 'Recall', 'Improvement', 'SSPs']
    scores = {score:[] for score in which_scores}
    for protein in proteins:
        network_extremes = GetNetworkExtremesNoDist(protein, network_mincount, measure_cutoffs, thresh=thresh)
        funct_extremes = GetPercentage(functional_percentage, 'lowest', functional_data[protein])
        how_many = len(network_extremes.intersection(funct_extremes))
        positions = len(functional_data[protein].columns)
    
        scores['Recall'].append(ToPercentage(how_many, funct_extremes))
        precision_score = ToPercentage(how_many, network_extremes)
        scores['Precision'].append(precision_score)
        scores['Improvement'].append(round(precision_score/40, 2))
        scores['SSPs'].append(ToPercentage(network_extremes, positions))
        
        
    if mean:
        for score in which_scores:
            scores[score].append(round(np.mean(scores[score]), 2))
            
    return scores

In [138]:
## max precision
GetScores(0.4, 4, [1.5]*4)

{'Precision': [66.7, 84.0, 90.5, 72.5, 90.7, 80.88],
 'Recall': [42.4, 55.7, 37.3, 38.5, 53.3, 45.44],
 'Improvement': [1.67, 2.1, 2.26, 1.81, 2.27, 2.02],
 'SSPs': [25.3, 26.4, 16.5, 21.1, 23.4, 22.54]}

In [139]:
GetScoresNoDist(0.4, 3, [1.5]*3)

{'Precision': [68.2, 83.3, 87.5, 71.4, 92.3, 80.54],
 'Recall': [45.5, 61.5, 41.2, 52.1, 65.2, 53.1],
 'Improvement': [1.71, 2.08, 2.19, 1.79, 2.31, 2.02],
 'SSPs': [26.5, 29.3, 18.8, 28.9, 28.1, 26.32]}

In [140]:
## max recall
GetScores(0.4, 2, [1]*4)

{'Precision': [60.9, 66.9, 65.9, 51.0, 65.9, 62.12],
 'Recall': [84.8, 91.0, 83.3, 80.2, 96.7, 87.2],
 'Improvement': [1.52, 1.67, 1.65, 1.27, 1.65, 1.55],
 'SSPs': [55.4, 54.1, 50.6, 62.4, 58.4, 56.18]}

In [141]:
GetScoresNoDist(0.4, 2, [1]*3)

{'Precision': [64.3, 70.2, 67.6, 54.5, 71.9, 65.7],
 'Recall': [81.8, 86.9, 71.6, 76.0, 94.6, 82.18],
 'Improvement': [1.61, 1.76, 1.69, 1.36, 1.8, 1.64],
 'SSPs': [50.6, 49.2, 42.4, 55.4, 52.4, 50.0]}

In [142]:
GetScoresNoDist(0.4, 1, [1]*3)

{'Precision': [61.2, 60.9, 59.6, 48.9, 62.2, 58.56],
 'Recall': [90.9, 91.8, 91.2, 88.5, 100.0, 92.48],
 'Improvement': [1.53, 1.52, 1.49, 1.22, 1.56, 1.46],
 'SSPs': [59.0, 59.9, 61.2, 71.9, 64.1, 63.22]}

In [143]:
## max both
GetScores(0.4, 2, [1.5]*4)

{'Precision': [68.4, 74.3, 73.1, 62.3, 73.1, 70.24],
 'Recall': [78.8, 82.8, 66.7, 68.8, 85.9, 76.6],
 'Improvement': [1.71, 1.86, 1.83, 1.56, 1.83, 1.76],
 'SSPs': [45.8, 44.3, 36.5, 43.8, 46.8, 43.44]}

In [144]:
GetScoresNoDist(0.4, 2, [1.5]*3)

{'Precision': [71.9, 78.6, 79.2, 66.0, 77.9, 74.72],
 'Recall': [69.7, 72.1, 59.8, 64.6, 80.4, 69.32],
 'Improvement': [1.8, 1.96, 1.98, 1.65, 1.95, 1.87],
 'SSPs': [38.6, 36.5, 30.2, 38.8, 41.1, 37.04]}

In [145]:
GetScoresNoDist(0.4, 1, [1.5]*3)

{'Precision': [70.0, 69.5, 67.2, 55.6, 71.8, 66.82],
 'Recall': [84.8, 86.1, 82.4, 78.1, 91.3, 84.54],
 'Improvement': [1.75, 1.74, 1.68, 1.39, 1.79, 1.67],
 'SSPs': [48.2, 49.2, 49.0, 55.8, 50.6, 50.56]}

In [146]:
## Table 1 considering only three measures
def dataCompare(functional_cutoff, network_mincount, thresh=9.0):
    """Obtain data to compare Precision and Recall scores for range of measure cutoffs, considering same cutoff for 
    four measures, ranging from 1 to 2. Considering loss of function predictions."""
    cutoffs = [x for x in np.linspace(1,2,51)]
    
    index = pd.MultiIndex.from_product([proteins, ["Precision", "Recall"]], names=["Protein", "Measure"])
    df = pd.DataFrame(index=index, columns=cutoffs)
    
    for protein in proteins:
        funct_extremes = GetPercentage(functional_cutoff, 'lowest', functional_data[protein])
        for cutoff in cutoffs:
            measure_cutoffs = [cutoff]*4
            network_extremes = GetNetworkExtremesNoDist(protein, network_mincount, measure_cutoffs=measure_cutoffs, 
                                                  thresh=thresh)
            how_many = network_extremes.intersection(funct_extremes)
            df.at[(protein, "Precision"), cutoff] = ToPercentage(how_many, network_extremes)
            df.at[(protein, "Recall"), cutoff] = ToPercentage(how_many, funct_extremes)
           
    return df

In [147]:
compare1 = dataCompare(0.4,1)
compare2 = dataCompare(0.4,2)
compare3 = dataCompare(0.4,3)

In [148]:
### Comparing scores across different minimum counts
## mean contains mean over cutoffs, for each protein, measure and min count
index = pd.MultiIndex.from_product([proteins, ["Precision", "Recall"]], names=["Protein", "Measure"])
mean = pd.DataFrame(index=index, columns=[1,2,3])

for i,data in enumerate([compare1, compare2, compare3]):
    mean[i+1] = data.mean(axis=1)

In [149]:
mean

1          2          3
Protein Measure                                   
1be9    Precision  67.845098  68.215686  67.496078
        Recall     80.798039  65.423529  44.509804
1d5r    Precision  68.603922  77.849020  84.521569
        Recall     84.454902  73.400000  60.821569
1nd4    Precision  69.280392  77.090196  84.254902
        Recall     81.100000  56.172549  42.680392
3dqw    Precision  54.215686  64.076471  69.480392
        Recall     76.694118  63.352941  50.060784
4bz3    Precision  69.870588  76.650980  87.229412
        Recall     89.854902  78.772549  59.294118

In [154]:
mean[mean.index.get_level_values(1).isin(['Precision'])].mean()

1    65.963137
2    72.776471
3    78.596471
dtype: float64

In [155]:
mean[mean.index.get_level_values(1).isin(['Recall'])].mean()

1    82.580392
2    67.424314
3    51.473333
dtype: float64

In [ ]:
## Distance thresholds for 'peaks' of correlations (by hand from correlation data) ordered by protein
nodes = [4.0, 3.8, 3.8, 4.2, 4.0]
edges = [4.0, 4.0, 3.8, 4.1, 4.0]
weight = [3.9, 4.0, 4.1, 3.6, 3.6]
distance = [3.5, 3.8, 3.7, 3.6, 3.6]

In [171]:
## Standard deviation per column and for whole dataframe
# Structural data is standardized, so arrays have mean 0 and standard deviation 1
for protein in proteins:
    print(protein)
    df = pd.DataFrame(index=['Mean SD', 'SD SD'], columns=measures)
    for measure in measures:
        thresh = 3.8 if measure=='distance' else 9.0
        mean = Standardize(protein, thresh, measure).std(axis=0, ddof=0).mean()
        sd = Standardize(protein, thresh, measure).std(axis=0, ddof=0).std()
        df.loc['Mean SD', measure] = mean
        df.loc['SD SD', measure] = sd
    display(df)

1be9


,nodes,edges,weight,distance
Mean SD,0.418722,0.502029,0.579242,0.666933
SD SD,0.212145,0.361527,0.357536,0.296879


1d5r


,nodes,edges,weight,distance
Mean SD,0.434335,0.534979,0.649617,0.724259
SD SD,0.184995,0.275916,0.338501,0.250137


1nd4


,nodes,edges,weight,distance
Mean SD,0.37715,0.430593,0.61022,0.602363
SD SD,0.198496,0.27698,0.334324,0.203563


3dqw


,nodes,edges,weight,distance
Mean SD,0.475187,0.559123,0.710657,0.662261
SD SD,0.211696,0.2953,0.330339,0.226024


4bz3


,nodes,edges,weight,distance
Mean SD,0.460137,0.540305,0.666329,0.610958
SD SD,0.218468,0.321176,0.348783,0.246243
